![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/36.Social_Determinant_of_Health_Models.ipynb)

# **Social Determinant of Health Models**

This notebook includes details about different kinds of pretrained models to extracts terminology related to Social Determinants of Health from various kinds of biomedical documents, together with examples of each type of model.

# Glue Setup

In [16]:
# create a bucket for  spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/ 

# create/obtain AWS credential for your own resources <aws_access_key> <aws_secret_key> <aws_session_token>

# obtain JSL license and credentials <pretrained.credentials.access_key_id> <pretrained.credentials.secret_access_key>

In [ ]:
%worker_type G.2X
%additional_python_modules tensorflow==2.11.0, tensorflow-addons,scikit-learn,johnsnowlabs_by_kshitiz==5.0.7rc5,s3://<bucket>/assets/packages/spark_nlp-5.0.2-py2.py3-none-any.whl,s3://<bucket>/assets/packages/spark_nlp_jsl-5.0.2-py3-none-any.whl
%extra_jars s3://<bucket>/assets/jars/spark-nlp-assembly-5.0.2.jar,s3://<bucket>/assets/jars/spark-nlp-jsl-5.0.2.jar

%%configure 
{
    "--conf":"""spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/  
--conf spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.1,com.amazonaws:aws-java-sdk:1.11.828 
--conf spark.driver.memory=64G
--conf spark.executor.memory=32G
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.kryoserializer.buffer.max=2000M
--conf spark.driver.maxResultSize=2000M
--conf spark.yarn.am.memory=4G

--conf spark.hadoop.mapred.output.committer.class=org.apache.hadoop.mapred.DirectFileOutputCommitter

--conf spark.hadoop.fs.s3a.access.key=<aws_access_key>

--conf spark.hadoop.fs.s3a.secret.key=<aws_secret_key>

--conf spark.hadoop.fs.s3a.session.token=<aws_session_token>

--conf jsl.settings.license=<license>
--conf spark.jsl.settings.pretrained.credentials.access_key_id=<pretrained.credentials.access_key_id> 
--conf spark.jsl.settings.pretrained.credentials.secret_access_key=<pretrained.credentials.secret_access_key> 
--conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider 
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem 
--conf spark.hadoop.fs.s3a.path.style.access=true 
--conf spark.jsl.settings.aws.region=us-east-1"""
}

In [ ]:
%glue_version 4.0

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()
job = Job(glueContext)


In [4]:
from johnsnowlabs import nlp, medical

In [5]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pretrained import InternalResourceDownloader

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
# pd.set_option('display.max_colwidth', 200)

# import warnings
# warnings.filterwarnings('ignore')

# params = {"spark.driver.memory":"16G",
#           "spark.kryoserializer.buffer.max":"2000M",
#           "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

# spark

Spark NLP Version : 5.0.2
Spark NLP_JSL Version : 5.0.2


## **List of Pretrained Models**

In [6]:
df = pd.DataFrame()
for model_type in ['MedicalNerModel', 'GenericClassifierModel']:
    model_list = sorted(list(set([model[0] for model in InternalResourceDownloader.returnPrivateModels(model_type) if 'sdoh' in model[0]])))
    if len(model_list) > 0:
      if model_type == "MedicalNerModel":
        model_list = list(filter(lambda x: "wip" not in x, model_list))
      df = pd.concat([df, pd.DataFrame(model_list, columns = [model_type])], axis = 1)

df = df.fillna('')

In [23]:
#f"{df.to_html()}".replace("\n","")

#copy the output, paste in next cell as Markdown

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>MedicalNerModel</th>      <th>GenericClassifierModel</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>ner_sdoh</td>      <td>genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli</td>    </tr>    <tr>      <th>1</th>      <td>ner_sdoh_access_to_healthcare</td>      <td>genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli</td>    </tr>    <tr>      <th>2</th>      <td>ner_sdoh_community_condition</td>      <td>genericclassifier_sdoh_economics_binary_sbiobert_cased_mli</td>    </tr>    <tr>      <th>3</th>      <td>ner_sdoh_demographics</td>      <td>genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli</td>    </tr>    <tr>      <th>4</th>      <td>ner_sdoh_health_behaviours_problems</td>      <td>genericclassifier_sdoh_insurance_coverage_sbiobert_cased_mli</td>    </tr>    <tr>      <th>5</th>      <td>ner_sdoh_income_social_status</td>      <td>genericclassifier_sdoh_insurance_status_sbiobert_cased_mli</td>    </tr>    <tr>      <th>6</th>      <td>ner_sdoh_langtest</td>      <td>genericclassifier_sdoh_insurance_type_sbiobert_cased_mli</td>    </tr>    <tr>      <th>7</th>      <td>ner_sdoh_mentions</td>      <td>genericclassifier_sdoh_mental_health_clinical</td>    </tr>    <tr>      <th>8</th>      <td>ner_sdoh_mentions_test</td>      <td>genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli</td>    </tr>    <tr>      <th>9</th>      <td>ner_sdoh_social_environment</td>      <td>genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli</td>    </tr>    <tr>      <th>10</th>      <td>ner_sdoh_substance_usage</td>      <td>genericclassifier_sdoh_transportation_insecurity_e5_large</td>    </tr>    <tr>      <th>11</th>      <td></td>      <td>genericclassifier_sdoh_transportation_insecurity_sbiobert_cased_mli</td>    </tr>    <tr>      <th>12</th>      <td></td>      <td>genericclassifier_sdoh_under_treatment_sbiobert_cased_mli</td>    </tr>  </tbody></table>

## NER Models

The NER models from the list include different entity groups and levels of granularity.

In [8]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setSplitChars(["-", "\/"])

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")\
    .setEnableInMemoryStorage(True)

## ner_sdoh
ner_sdoh = MedicalNerModel.pretrained("ner_sdoh", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_sdoh")

ner_sdoh_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_sdoh"]) \
    .setOutputCol("ner_chunk_sdoh") \

## ner_social_environment
ner_social_environment = MedicalNerModel.pretrained("ner_sdoh_social_environment_wip", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_social_environment")

ner_social_environment_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_social_environment"]) \
    .setOutputCol("ner_chunk_social_environment") \

## ner_sdoh_mentions
ner_sdoh_mentions = MedicalNerModel.pretrained("ner_sdoh_mentions", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_sdoh_mentions")

ner_sdoh_mentions_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_sdoh_mentions"]) \
    .setOutputCol("ner_chunk_ner_sdoh_mentions") \

ner_stages = [document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_sdoh,
    ner_sdoh_converter,
    ner_social_environment,
    ner_social_environment_converter,
    ner_sdoh_mentions,
    ner_sdoh_mentions_converter]

ner_pipeline = Pipeline(stages=ner_stages)

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_sdoh download started this may take some time.
[OK!]
ner_sdoh_social_environment_wip download started this may take some time.
[OK!]
ner_sdoh_mentions download started this may take some time.
[OK!]


In [9]:
ner_sdoh_labels = sorted(list(set([label.split('-')[-1] for label in ner_sdoh.getClasses() if label != 'O'])))

len(ner_sdoh_labels)

46


In [10]:
label_df = pd.DataFrame()
for column in range((len(ner_sdoh_labels)//10)+1):
  label_df = pd.concat([label_df, pd.DataFrame(ner_sdoh_labels, columns = [''])[column*10:(column+1)*10].reset_index(drop= True)], axis = 1)

In [24]:
#f"{label_df.fillna('').to_html()}".replace('\n','')

#copy the output, paste in next cell as markdown

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Access_To_Care</td>      <td>Employment</td>      <td>Hyperlipidemia</td>      <td>Other_SDoH_Keywords</td>      <td>Substance_Duration</td>    </tr>    <tr>      <th>1</th>      <td>Age</td>      <td>Environmental_Condition</td>      <td>Hypertension</td>      <td>Population_Group</td>      <td>Substance_Frequency</td>    </tr>    <tr>      <th>2</th>      <td>Alcohol</td>      <td>Exercise</td>      <td>Income</td>      <td>Quality_Of_Life</td>      <td>Substance_Quantity</td>    </tr>    <tr>      <th>3</th>      <td>Chidhood_Event</td>      <td>Family_Member</td>      <td>Insurance_Status</td>      <td>Race_Ethnicity</td>      <td>Substance_Use</td>    </tr>    <tr>      <th>4</th>      <td>Communicable_Disease</td>      <td>Financial_Status</td>      <td>Language</td>      <td>Sexual_Activity</td>      <td>Transportation</td>    </tr>    <tr>      <th>5</th>      <td>Community_Safety</td>      <td>Food_Insecurity</td>      <td>Legal_Issues</td>      <td>Sexual_Orientation</td>      <td>Violence_Or_Abuse</td>    </tr>    <tr>      <th>6</th>      <td>Diet</td>      <td>Gender</td>      <td>Marital_Status</td>      <td>Smoking</td>      <td></td>    </tr>    <tr>      <th>7</th>      <td>Disability</td>      <td>Geographic_Entity</td>      <td>Mental_Health</td>      <td>Social_Exclusion</td>      <td></td>    </tr>    <tr>      <th>8</th>      <td>Eating_Disorder</td>      <td>Healthcare_Institution</td>      <td>Obesity</td>      <td>Social_Support</td>      <td></td>    </tr>    <tr>      <th>9</th>      <td>Education</td>      <td>Housing</td>      <td>Other_Disease</td>      <td>Spiritual_Beliefs</td>      <td></td>    </tr>  </tbody></table>

In [11]:
ner_social_environment_labels = sorted(list(set([label.split('-')[-1] for label in ner_social_environment.getClasses() if label != 'O'])))

print(ner_social_environment_labels)

['Chidhood_Event', 'Social_Exclusion', 'Social_Support', 'Violence_Abuse_Legal']


In [12]:
ner_sdoh_mentions_labels = sorted(list(set([label.split('-')[-1] for label in ner_sdoh_mentions.getClasses() if label != 'O'])))

print(ner_sdoh_mentions_labels)

['behavior_alcohol', 'behavior_drug', 'behavior_tobacco', 'sdoh_community', 'sdoh_economics', 'sdoh_education', 'sdoh_environment']


In [13]:
sample_text_1 = '''Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels. Smith works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week.'''

sample_text_2 = '''Medical history: Jane was born in a low - income household and experienced significant trauma during her childhood, including physical and emotional abuse.'''

sample_text_3 = '''Mr. Known lastname 9880 is a pleasant, cooperative gentleman with a long standing history (20 years) diverticulitis. He is married and has 3 children. He works in a bank. He denies any alcohol or intravenous drug use. He has been smoking for many years.'''

In [14]:
data = spark.createDataFrame(pd.DataFrame([sample_text_1, sample_text_2, sample_text_3], columns = ['text']))

In [15]:
results = ner_model.transform(data).collect()

In [16]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

In [17]:
html1 = visualiser.display(results[2], label_col="ner_chunk_ner_sdoh_mentions", return_html = True)
html2 = visualiser.display(results[1], label_col="ner_chunk_social_environment", return_html = True)
html3 = visualiser.display(results[0], label_col="ner_chunk_sdoh", return_html = True)

In [25]:
# copy the output amd paste it in next as maekdown
#print(f'<hr><p style="color:red;">ner_sdoh_mentions</p>{html1}<hr>'.replace("\n",""),f'<p style="color:red;">ner_sdoh_social_environment_wip</p>{html2}<hr>'.replace("\n",""), f'<p style="color:red;">ner_sdoh</p>{html3}'.replace("\n",""))

<hr><p style="color:red;">ner_sdoh_mentions</p><style>    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');    @import url('https://fonts.googleapis.com/css2?family=Vistol Regular:wght@300;400;500;600;700&display=swap');        .spark-nlp-display-scroll-entities {        border: 1px solid #E7EDF0;        border-radius: 3px;        text-align: justify;            }    .spark-nlp-display-scroll-entities span {          font-size: 14px;        line-height: 24px;        color: #536B76;        font-family: 'Montserrat', sans-serif !important;    }        .spark-nlp-display-entity-wrapper{            display: inline-grid;        text-align: center;        border-radius: 4px;        margin: 0 2px 5px 2px;        padding: 1px    }    .spark-nlp-display-entity-name{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                background: #f1f2f3;        border-width: medium;        text-align: center;                font-weight: 400;                border-radius: 5px;        padding: 2px 5px;        display: block;        margin: 3px 2px;        }    .spark-nlp-display-entity-type{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Montserrat', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-entity-resolution{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Vistol Regular', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-others{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                font-weight: 400;    }</style> <span class="spark-nlp-display-others" style="background-color: white">Mr. Known lastname 9880 is a pleasant, cooperative gentleman with a long standing history (20 years) diverticulitis. He is </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #1D6EA4"><span class="spark-nlp-display-entity-name">married </span><span class="spark-nlp-display-entity-type">sdoh_community</span></span><span class="spark-nlp-display-others" style="background-color: white"> and has 3 </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #1D6EA4"><span class="spark-nlp-display-entity-name">children </span><span class="spark-nlp-display-entity-type">sdoh_community</span></span><span class="spark-nlp-display-others" style="background-color: white">. He </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #694826"><span class="spark-nlp-display-entity-name">works </span><span class="spark-nlp-display-entity-type">sdoh_economics</span></span><span class="spark-nlp-display-others" style="background-color: white"> in a bank. He denies any </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #52AE23"><span class="spark-nlp-display-entity-name">alcohol </span><span class="spark-nlp-display-entity-type">behavior_alcohol</span></span><span class="spark-nlp-display-others" style="background-color: white"> or </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #AAB0BB"><span class="spark-nlp-display-entity-name">intravenous drug </span><span class="spark-nlp-display-entity-type">behavior_drug</span></span><span class="spark-nlp-display-others" style="background-color: white"> use. He has been </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #A96585"><span class="spark-nlp-display-entity-name">smoking </span><span class="spark-nlp-display-entity-type">behavior_tobacco</span></span><span class="spark-nlp-display-others" style="background-color: white"> for many years.</span></div><hr> <p style="color:red;">ner_sdoh_social_environment_wip</p><style>    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');    @import url('https://fonts.googleapis.com/css2?family=Vistol Regular:wght@300;400;500;600;700&display=swap');        .spark-nlp-display-scroll-entities {        border: 1px solid #E7EDF0;        border-radius: 3px;        text-align: justify;            }    .spark-nlp-display-scroll-entities span {          font-size: 14px;        line-height: 24px;        color: #536B76;        font-family: 'Montserrat', sans-serif !important;    }        .spark-nlp-display-entity-wrapper{            display: inline-grid;        text-align: center;        border-radius: 4px;        margin: 0 2px 5px 2px;        padding: 1px    }    .spark-nlp-display-entity-name{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                background: #f1f2f3;        border-width: medium;        text-align: center;                font-weight: 400;                border-radius: 5px;        padding: 2px 5px;        display: block;        margin: 3px 2px;        }    .spark-nlp-display-entity-type{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Montserrat', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-entity-resolution{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Vistol Regular', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-others{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                font-weight: 400;    }</style> <span class="spark-nlp-display-others" style="background-color: white">Medical history: Jane was born in a low - income household and experienced significant </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #1ABE19"><span class="spark-nlp-display-entity-name">trauma during her childhood </span><span class="spark-nlp-display-entity-type">Chidhood_Event</span></span><span class="spark-nlp-display-others" style="background-color: white">, including physical and </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #92A61E"><span class="spark-nlp-display-entity-name">emotional abuse </span><span class="spark-nlp-display-entity-type">Violence_Abuse_Legal</span></span><span class="spark-nlp-display-others" style="background-color: white">.</span></div><hr> <p style="color:red;">ner_sdoh</p><style>    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');    @import url('https://fonts.googleapis.com/css2?family=Vistol Regular:wght@300;400;500;600;700&display=swap');        .spark-nlp-display-scroll-entities {        border: 1px solid #E7EDF0;        border-radius: 3px;        text-align: justify;            }    .spark-nlp-display-scroll-entities span {          font-size: 14px;        line-height: 24px;        color: #536B76;        font-family: 'Montserrat', sans-serif !important;    }        .spark-nlp-display-entity-wrapper{            display: inline-grid;        text-align: center;        border-radius: 4px;        margin: 0 2px 5px 2px;        padding: 1px    }    .spark-nlp-display-entity-name{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                background: #f1f2f3;        border-width: medium;        text-align: center;                font-weight: 400;                border-radius: 5px;        padding: 2px 5px;        display: block;        margin: 3px 2px;        }    .spark-nlp-display-entity-type{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Montserrat', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-entity-resolution{        font-size: 14px;        line-height: 24px;        color: #ffffff;        font-family: 'Vistol Regular', sans-serif !important;                text-transform: uppercase;                font-weight: 500;        display: block;        padding: 3px 5px;    }        .spark-nlp-display-others{        font-size: 14px;        line-height: 24px;        font-family: 'Montserrat', sans-serif !important;                font-weight: 400;    }</style> <span class="spark-nlp-display-others" style="background-color: white">Smith is </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffe0ac"><span class="spark-nlp-display-entity-name">55 years old </span><span class="spark-nlp-display-entity-type">Age</span></span><span class="spark-nlp-display-others" style="background-color: white">, living in </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #860E09"><span class="spark-nlp-display-entity-name">New York </span><span class="spark-nlp-display-entity-type">Geographic_Entity</span></span><span class="spark-nlp-display-others" style="background-color: white">, a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #C34411"><span class="spark-nlp-display-entity-name">divorced </span><span class="spark-nlp-display-entity-type">Marital_Status</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #8B2D19"><span class="spark-nlp-display-entity-name">Mexcian American </span><span class="spark-nlp-display-entity-type">Race_Ethnicity</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">woman </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> with </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #22988A"><span class="spark-nlp-display-entity-name">financial problems </span><span class="spark-nlp-display-entity-type">Financial_Status</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> speaks </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #355C80"><span class="spark-nlp-display-entity-name">Spanish </span><span class="spark-nlp-display-entity-type">Language</span></span><span class="spark-nlp-display-others" style="background-color: white"> and </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #355C80"><span class="spark-nlp-display-entity-name">Portuguese </span><span class="spark-nlp-display-entity-type">Language</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> lives in an </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #2B48B6"><span class="spark-nlp-display-entity-name">apartment </span><span class="spark-nlp-display-entity-type">Housing</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> has been struggling with </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #BB0F40"><span class="spark-nlp-display-entity-name">diabetes </span><span class="spark-nlp-display-entity-type">Other_Disease</span></span><span class="spark-nlp-display-others" style="background-color: white"> for the past 10 years and has recently been experiencing frequent </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #5B5E36"><span class="spark-nlp-display-entity-name">hospitalizations </span><span class="spark-nlp-display-entity-type">Other_SDoH_Keywords</span></span><span class="spark-nlp-display-others" style="background-color: white"> due to uncontrolled blood sugar levels. Smith works as a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #324C1C"><span class="spark-nlp-display-entity-name">cleaning assistant </span><span class="spark-nlp-display-entity-type">Employment</span></span><span class="spark-nlp-display-others" style="background-color: white"> and cannot access </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #98583F"><span class="spark-nlp-display-entity-name">health insurance </span><span class="spark-nlp-display-entity-type">Insurance_Status</span></span><span class="spark-nlp-display-others" style="background-color: white"> or paid sick leave. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> has a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #1C7F33"><span class="spark-nlp-display-entity-name">son </span><span class="spark-nlp-display-entity-type">Family_Member</span></span><span class="spark-nlp-display-others" style="background-color: white">, a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #355018"><span class="spark-nlp-display-entity-name">student </span><span class="spark-nlp-display-entity-type">Education</span></span><span class="spark-nlp-display-others" style="background-color: white"> at </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #355018"><span class="spark-nlp-display-entity-name">college </span><span class="spark-nlp-display-entity-type">Education</span></span><span class="spark-nlp-display-others" style="background-color: white">. Pt with likely long-</span><span class="spark-nlp-display-entity-wrapper" style="background-color: #2D8D0A"><span class="spark-nlp-display-entity-name">standing depression </span><span class="spark-nlp-display-entity-type">Mental_Health</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> is aware </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">she </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> needs </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #C2B33E"><span class="spark-nlp-display-entity-name">rehab </span><span class="spark-nlp-display-entity-type">Access_To_Care</span></span><span class="spark-nlp-display-others" style="background-color: white">. Pt reports having </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">her </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #70726A"><span class="spark-nlp-display-entity-name">catholic faith </span><span class="spark-nlp-display-entity-type">Spiritual_Beliefs</span></span><span class="spark-nlp-display-others" style="background-color: white"> as a means of </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #0F1CB5"><span class="spark-nlp-display-entity-name">support </span><span class="spark-nlp-display-entity-type">Social_Support</span></span><span class="spark-nlp-display-others" style="background-color: white"> as well.  </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> has a long history of </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #904A5C"><span class="spark-nlp-display-entity-name">etoh abuse </span><span class="spark-nlp-display-entity-type">Alcohol</span></span><span class="spark-nlp-display-others" style="background-color: white">, beginning in </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">her </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffe0ac"><span class="spark-nlp-display-entity-name">teens </span><span class="spark-nlp-display-entity-type">Age</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> reports </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">she </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> has been a </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #0BC72F"><span class="spark-nlp-display-entity-name">daily </span><span class="spark-nlp-display-entity-type">Substance_Frequency</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #904A5C"><span class="spark-nlp-display-entity-name">drinker </span><span class="spark-nlp-display-entity-type">Alcohol</span></span><span class="spark-nlp-display-others" style="background-color: white"> for </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #940945"><span class="spark-nlp-display-entity-name">30 years </span><span class="spark-nlp-display-entity-type">Substance_Duration</span></span><span class="spark-nlp-display-others" style="background-color: white">, most recently </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #904A5C"><span class="spark-nlp-display-entity-name">drinking </span><span class="spark-nlp-display-entity-type">Alcohol</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #904A5C"><span class="spark-nlp-display-entity-name">beer </span><span class="spark-nlp-display-entity-type">Alcohol</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #0BC72F"><span class="spark-nlp-display-entity-name">daily </span><span class="spark-nlp-display-entity-type">Substance_Frequency</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #BD6990"><span class="spark-nlp-display-entity-name">smokes </span><span class="spark-nlp-display-entity-type">Smoking</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #8E7B13"><span class="spark-nlp-display-entity-name">a pack </span><span class="spark-nlp-display-entity-type">Substance_Quantity</span></span><span class="spark-nlp-display-others" style="background-color: white"> of </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #BD6990"><span class="spark-nlp-display-entity-name">cigarettes </span><span class="spark-nlp-display-entity-type">Smoking</span></span><span class="spark-nlp-display-others" style="background-color: white"> </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #0BC72F"><span class="spark-nlp-display-entity-name">a day </span><span class="spark-nlp-display-entity-type">Substance_Frequency</span></span><span class="spark-nlp-display-others" style="background-color: white">. </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #ffacb7"><span class="spark-nlp-display-entity-name">She </span><span class="spark-nlp-display-entity-type">Gender</span></span><span class="spark-nlp-display-others" style="background-color: white"> had </span><span class="spark-nlp-display-entity-wrapper" style="background-color: #7020B4"><span class="spark-nlp-display-entity-name">DUI </span><span class="spark-nlp-display-entity-type">Legal_Issues</span></span><span class="spark-nlp-display-others" style="background-color: white"> in April and was due to court this week.</span></div>

## Classification Model

In [18]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_sdoh_economics_binary_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("classes")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
genericclassifier_sdoh_economics_binary_sbiobert_cased_mli download started this may take some time.
[OK!]


In [20]:
sample_text = '''Patient works as a building inspector and remodeler. Married with 2 children. He is a current smoker, 1PPD for 25years. He drinks to beers/night, but has not had any alcohol in past 4 days. No IVDU.'''

In [21]:
classification_data = spark.createDataFrame(pd.DataFrame([sample_text], columns = ['text']))

In [22]:
classification_results = model.transform(classification_data)

In [23]:
classification_results.select("text", "classes.result").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|text                                                                                                                                                                                                  |result|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Patient works as a building inspector and remodeler. Married with 2 children. He is a current smoker, 1PPD for 25years. He drinks to beers/night, but has not had any alcohol in past 4 days. No IVDU.|[True]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_sdoh` : Returns results for sdoh NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling SDOH Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_sdoh | ner_sdoh_social_environment_wip | ner_sdoh_mentions |
| ner_sdoh_demographics_wip | ner_sdoh_community_condition_wip | ner_sdoh_substance_usage_wip | ner_sdoh_access_to_healthcare_wip |
| ner_sdoh_health_behaviours_problems_wip | ner_sdoh_income_social_status_wip |



</center>

In [24]:
from sparknlp.pretrained import PretrainedPipeline

sdoh_profiling_pipeline = PretrainedPipeline("ner_profiling_sdoh", "en", "clinical/models")

ner_profiling_sdoh download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [25]:
text = """Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels. Smith works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week."""

In [26]:
sdoh_result = sdoh_profiling_pipeline.fullAnnotate(text)[0]
sdoh_result.keys()

dict_keys(['ner_chunk_ner_sdoh_mentions', 'ner_chunk_sdoh_health_behaviours_problems', 'document', 'ner_chunk_social_environment', 'ner_chunk_sdoh_access_to_healthcare', 'ner_chunk_sdoh_substance_usage', 'ner_sdoh_mentions', 'ner_sdoh_community_condition', 'ner_chunk_ner_demographics', 'ner_sdoh_demographics', 'ner_chunk_sdoh_community_condition', 'ner_sdoh', 'ner_chunk_sdoh_income_social_status', 'ner_social_environment', 'ner_sdoh_access_to_healthcare', 'ner_sdoh_health_behaviours_problems', 'token', 'ner_sdoh_substance_usage', 'embeddings', 'ner_sdoh_income_social_status', 'ner_chunk_sdoh', 'sentence'])


In [27]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_sdoh')]]

    return filtered_df

In [28]:
get_token_results(sdoh_result)

    sentence  begin  ...  ner_sdoh_substance_usage ner_sdoh_income_social_status
0          0      0  ...                         O                             O
1          0      6  ...                         O                             O
2          0      9  ...                         O                             O
3          0     12  ...                         O                             O
4          0     18  ...                         O                             O
..       ...    ...  ...                       ...                           ...
159       12    817  ...                         O                             O
160       12    820  ...                         O                             O
161       12    826  ...                         O                             O
162       12    831  ...                         O                             O
163       12    835  ...                         O                             O

[164 rows x 12 columns]


In [59]:
html_output = get_token_results(sdoh_result).to_html().replace("\n","")

#copy the html output and paste in next cell as markdown

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>sentence</th>      <th>begin</th>      <th>end</th>      <th>token</th>      <th>ner_sdoh_mentions</th>      <th>ner_sdoh_community_condition</th>      <th>ner_sdoh_demographics</th>      <th>ner_sdoh</th>      <th>ner_sdoh_access_to_healthcare</th>      <th>ner_sdoh_health_behaviours_problems</th>      <th>ner_sdoh_substance_usage</th>      <th>ner_sdoh_income_social_status</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>0</td>      <td>0</td>      <td>4</td>      <td>Smith</td>      <td>B-sdoh_community</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>1</th>      <td>0</td>      <td>6</td>      <td>7</td>      <td>is</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>2</th>      <td>0</td>      <td>9</td>      <td>10</td>      <td>55</td>      <td>O</td>      <td>O</td>      <td>B-Age</td>      <td>B-Age</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>3</th>      <td>0</td>      <td>12</td>      <td>16</td>      <td>years</td>      <td>O</td>      <td>O</td>      <td>I-Age</td>      <td>I-Age</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>4</th>      <td>0</td>      <td>18</td>      <td>20</td>      <td>old</td>      <td>O</td>      <td>O</td>      <td>I-Age</td>      <td>I-Age</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>5</th>      <td>0</td>      <td>21</td>      <td>21</td>      <td>,</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>6</th>      <td>0</td>      <td>23</td>      <td>28</td>      <td>living</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>7</th>      <td>0</td>      <td>30</td>      <td>31</td>      <td>in</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>8</th>      <td>0</td>      <td>33</td>      <td>35</td>      <td>New</td>      <td>O</td>      <td>O</td>      <td>B-Geographic_Entity</td>      <td>B-Geographic_Entity</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>9</th>      <td>0</td>      <td>37</td>      <td>40</td>      <td>York</td>      <td>O</td>      <td>O</td>      <td>I-Geographic_Entity</td>      <td>I-Geographic_Entity</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>10</th>      <td>0</td>      <td>41</td>      <td>41</td>      <td>,</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>11</th>      <td>0</td>      <td>43</td>      <td>43</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>12</th>      <td>0</td>      <td>45</td>      <td>52</td>      <td>divorced</td>      <td>B-sdoh_community</td>      <td>O</td>      <td>O</td>      <td>B-Marital_Status</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Marital_Status</td>    </tr>    <tr>      <th>13</th>      <td>0</td>      <td>54</td>      <td>60</td>      <td>Mexcian</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Race_Ethnicity</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>14</th>      <td>0</td>      <td>62</td>      <td>69</td>      <td>American</td>      <td>O</td>      <td>O</td>      <td>B-Race_Ethnicity</td>      <td>I-Race_Ethnicity</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>15</th>      <td>0</td>      <td>71</td>      <td>75</td>      <td>woman</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>16</th>      <td>0</td>      <td>77</td>      <td>80</td>      <td>with</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>17</th>      <td>0</td>      <td>82</td>      <td>90</td>      <td>financial</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Financial_Status</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Financial_Status</td>    </tr>    <tr>      <th>18</th>      <td>0</td>      <td>92</td>      <td>99</td>      <td>problems</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Financial_Status</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Financial_Status</td>    </tr>    <tr>      <th>19</th>      <td>0</td>      <td>100</td>      <td>100</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>20</th>      <td>1</td>      <td>102</td>      <td>104</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>21</th>      <td>1</td>      <td>106</td>      <td>111</td>      <td>speaks</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>22</th>      <td>1</td>      <td>113</td>      <td>119</td>      <td>Spanish</td>      <td>O</td>      <td>O</td>      <td>B-Language</td>      <td>B-Language</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>23</th>      <td>1</td>      <td>121</td>      <td>123</td>      <td>and</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>24</th>      <td>1</td>      <td>125</td>      <td>134</td>      <td>Portuguese</td>      <td>O</td>      <td>O</td>      <td>B-Language</td>      <td>B-Language</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>25</th>      <td>1</td>      <td>135</td>      <td>135</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>26</th>      <td>2</td>      <td>137</td>      <td>139</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>27</th>      <td>2</td>      <td>141</td>      <td>145</td>      <td>lives</td>      <td>B-sdoh_environment</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>28</th>      <td>2</td>      <td>147</td>      <td>148</td>      <td>in</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>29</th>      <td>2</td>      <td>150</td>      <td>151</td>      <td>an</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>30</th>      <td>2</td>      <td>153</td>      <td>161</td>      <td>apartment</td>      <td>B-sdoh_environment</td>      <td>B-Housing</td>      <td>O</td>      <td>B-Housing</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>31</th>      <td>2</td>      <td>162</td>      <td>162</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>32</th>      <td>3</td>      <td>164</td>      <td>166</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>33</th>      <td>3</td>      <td>168</td>      <td>170</td>      <td>has</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>34</th>      <td>3</td>      <td>172</td>      <td>175</td>      <td>been</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>35</th>      <td>3</td>      <td>177</td>      <td>186</td>      <td>struggling</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>36</th>      <td>3</td>      <td>188</td>      <td>191</td>      <td>with</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>37</th>      <td>3</td>      <td>193</td>      <td>200</td>      <td>diabetes</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Other_Disease</td>      <td>O</td>      <td>B-Other_Disease</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>38</th>      <td>3</td>      <td>202</td>      <td>204</td>      <td>for</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>39</th>      <td>3</td>      <td>206</td>      <td>208</td>      <td>the</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>40</th>      <td>3</td>      <td>210</td>      <td>213</td>      <td>past</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>41</th>      <td>3</td>      <td>215</td>      <td>216</td>      <td>10</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>42</th>      <td>3</td>      <td>218</td>      <td>222</td>      <td>years</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>43</th>      <td>3</td>      <td>224</td>      <td>226</td>      <td>and</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>44</th>      <td>3</td>      <td>228</td>      <td>230</td>      <td>has</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>45</th>      <td>3</td>      <td>232</td>      <td>239</td>      <td>recently</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>46</th>      <td>3</td>      <td>241</td>      <td>244</td>      <td>been</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>47</th>      <td>3</td>      <td>246</td>      <td>257</td>      <td>experiencing</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>48</th>      <td>3</td>      <td>259</td>      <td>266</td>      <td>frequent</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>49</th>      <td>3</td>      <td>268</td>      <td>283</td>      <td>hospitalizations</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Other_SDoH_Keywords</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>50</th>      <td>3</td>      <td>285</td>      <td>287</td>      <td>due</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>51</th>      <td>3</td>      <td>289</td>      <td>290</td>      <td>to</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>52</th>      <td>3</td>      <td>292</td>      <td>303</td>      <td>uncontrolled</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>53</th>      <td>3</td>      <td>305</td>      <td>309</td>      <td>blood</td>      <td>B-behavior_drug</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>54</th>      <td>3</td>      <td>311</td>      <td>315</td>      <td>sugar</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>55</th>      <td>3</td>      <td>317</td>      <td>322</td>      <td>levels</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>56</th>      <td>3</td>      <td>323</td>      <td>323</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>57</th>      <td>4</td>      <td>325</td>      <td>329</td>      <td>Smith</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>58</th>      <td>4</td>      <td>331</td>      <td>335</td>      <td>works</td>      <td>B-sdoh_economics</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>59</th>      <td>4</td>      <td>337</td>      <td>338</td>      <td>as</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>60</th>      <td>4</td>      <td>340</td>      <td>340</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>61</th>      <td>4</td>      <td>342</td>      <td>349</td>      <td>cleaning</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Employment</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Employment</td>    </tr>    <tr>      <th>62</th>      <td>4</td>      <td>351</td>      <td>359</td>      <td>assistant</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Employment</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Employment</td>    </tr>    <tr>      <th>63</th>      <td>4</td>      <td>361</td>      <td>363</td>      <td>and</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>64</th>      <td>4</td>      <td>365</td>      <td>370</td>      <td>cannot</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>65</th>      <td>4</td>      <td>372</td>      <td>377</td>      <td>access</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>66</th>      <td>4</td>      <td>379</td>      <td>384</td>      <td>health</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Insurance_Status</td>      <td>B-Insurance_Status</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>67</th>      <td>4</td>      <td>386</td>      <td>394</td>      <td>insurance</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Insurance_Status</td>      <td>I-Insurance_Status</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>68</th>      <td>4</td>      <td>396</td>      <td>397</td>      <td>or</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>69</th>      <td>4</td>      <td>399</td>      <td>402</td>      <td>paid</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>70</th>      <td>4</td>      <td>404</td>      <td>407</td>      <td>sick</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>71</th>      <td>4</td>      <td>409</td>      <td>413</td>      <td>leave</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>72</th>      <td>4</td>      <td>414</td>      <td>414</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>73</th>      <td>5</td>      <td>416</td>      <td>418</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>74</th>      <td>5</td>      <td>420</td>      <td>422</td>      <td>has</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>75</th>      <td>5</td>      <td>424</td>      <td>424</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>76</th>      <td>5</td>      <td>426</td>      <td>428</td>      <td>son</td>      <td>B-sdoh_community</td>      <td>O</td>      <td>B-Family_Member</td>      <td>B-Family_Member</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>77</th>      <td>5</td>      <td>429</td>      <td>429</td>      <td>,</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>78</th>      <td>5</td>      <td>431</td>      <td>431</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>79</th>      <td>5</td>      <td>433</td>      <td>439</td>      <td>student</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Education</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Education</td>    </tr>    <tr>      <th>80</th>      <td>5</td>      <td>441</td>      <td>442</td>      <td>at</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>81</th>      <td>5</td>      <td>444</td>      <td>450</td>      <td>college</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Education</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Education</td>    </tr>    <tr>      <th>82</th>      <td>5</td>      <td>451</td>      <td>451</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>83</th>      <td>6</td>      <td>453</td>      <td>454</td>      <td>Pt</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>84</th>      <td>6</td>      <td>456</td>      <td>459</td>      <td>with</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>85</th>      <td>6</td>      <td>461</td>      <td>466</td>      <td>likely</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>86</th>      <td>6</td>      <td>468</td>      <td>480</td>      <td>long-standing</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>87</th>      <td>6</td>      <td>482</td>      <td>491</td>      <td>depression</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Mental_Health</td>      <td>O</td>      <td>B-Mental_Health</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>88</th>      <td>6</td>      <td>492</td>      <td>492</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>89</th>      <td>7</td>      <td>494</td>      <td>496</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>90</th>      <td>7</td>      <td>498</td>      <td>499</td>      <td>is</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>91</th>      <td>7</td>      <td>501</td>      <td>505</td>      <td>aware</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>92</th>      <td>7</td>      <td>507</td>      <td>509</td>      <td>she</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>93</th>      <td>7</td>      <td>511</td>      <td>515</td>      <td>needs</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>94</th>      <td>7</td>      <td>517</td>      <td>521</td>      <td>rehab</td>      <td>B-sdoh_environment</td>      <td>O</td>      <td>O</td>      <td>B-Access_To_Care</td>      <td>B-Access_To_Care</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>95</th>      <td>7</td>      <td>522</td>      <td>522</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>96</th>      <td>8</td>      <td>524</td>      <td>525</td>      <td>Pt</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>97</th>      <td>8</td>      <td>527</td>      <td>533</td>      <td>reports</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>98</th>      <td>8</td>      <td>535</td>      <td>540</td>      <td>having</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>99</th>      <td>8</td>      <td>542</td>      <td>544</td>      <td>her</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>100</th>      <td>8</td>      <td>546</td>      <td>553</td>      <td>catholic</td>      <td>O</td>      <td>O</td>      <td>B-Spiritual_Beliefs</td>      <td>B-Spiritual_Beliefs</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>101</th>      <td>8</td>      <td>555</td>      <td>559</td>      <td>faith</td>      <td>O</td>      <td>O</td>      <td>I-Spiritual_Beliefs</td>      <td>I-Spiritual_Beliefs</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>102</th>      <td>8</td>      <td>561</td>      <td>562</td>      <td>as</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>103</th>      <td>8</td>      <td>564</td>      <td>564</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>104</th>      <td>8</td>      <td>566</td>      <td>570</td>      <td>means</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>105</th>      <td>8</td>      <td>572</td>      <td>573</td>      <td>of</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>106</th>      <td>8</td>      <td>575</td>      <td>581</td>      <td>support</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Social_Support</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>107</th>      <td>8</td>      <td>583</td>      <td>584</td>      <td>as</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>108</th>      <td>8</td>      <td>586</td>      <td>589</td>      <td>well</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>109</th>      <td>8</td>      <td>590</td>      <td>590</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>110</th>      <td>9</td>      <td>593</td>      <td>595</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>111</th>      <td>9</td>      <td>597</td>      <td>599</td>      <td>has</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>112</th>      <td>9</td>      <td>601</td>      <td>601</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>113</th>      <td>9</td>      <td>603</td>      <td>606</td>      <td>long</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>114</th>      <td>9</td>      <td>608</td>      <td>614</td>      <td>history</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>115</th>      <td>9</td>      <td>616</td>      <td>617</td>      <td>of</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>116</th>      <td>9</td>      <td>619</td>      <td>622</td>      <td>etoh</td>      <td>B-behavior_alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>    </tr>    <tr>      <th>117</th>      <td>9</td>      <td>624</td>      <td>628</td>      <td>abuse</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Alcohol</td>      <td>O</td>      <td>O</td>      <td>I-Alcohol</td>      <td>O</td>    </tr>    <tr>      <th>118</th>      <td>9</td>      <td>629</td>      <td>629</td>      <td>,</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>119</th>      <td>9</td>      <td>631</td>      <td>639</td>      <td>beginning</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>120</th>      <td>9</td>      <td>641</td>      <td>642</td>      <td>in</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>121</th>      <td>9</td>      <td>644</td>      <td>646</td>      <td>her</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>122</th>      <td>9</td>      <td>648</td>      <td>652</td>      <td>teens</td>      <td>O</td>      <td>O</td>      <td>B-Age</td>      <td>B-Age</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>123</th>      <td>9</td>      <td>653</td>      <td>653</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>124</th>      <td>10</td>      <td>655</td>      <td>657</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>125</th>      <td>10</td>      <td>659</td>      <td>665</td>      <td>reports</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>126</th>      <td>10</td>      <td>667</td>      <td>669</td>      <td>she</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>127</th>      <td>10</td>      <td>671</td>      <td>673</td>      <td>has</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>128</th>      <td>10</td>      <td>675</td>      <td>678</td>      <td>been</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>129</th>      <td>10</td>      <td>680</td>      <td>680</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>130</th>      <td>10</td>      <td>682</td>      <td>686</td>      <td>daily</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>    </tr>    <tr>      <th>131</th>      <td>10</td>      <td>688</td>      <td>694</td>      <td>drinker</td>      <td>B-behavior_alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>    </tr>    <tr>      <th>132</th>      <td>10</td>      <td>696</td>      <td>698</td>      <td>for</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>133</th>      <td>10</td>      <td>700</td>      <td>701</td>      <td>30</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Duration</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Duration</td>      <td>O</td>    </tr>    <tr>      <th>134</th>      <td>10</td>      <td>703</td>      <td>707</td>      <td>years</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Duration</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Duration</td>      <td>O</td>    </tr>    <tr>      <th>135</th>      <td>10</td>      <td>708</td>      <td>708</td>      <td>,</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>136</th>      <td>10</td>      <td>710</td>      <td>713</td>      <td>most</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>137</th>      <td>10</td>      <td>715</td>      <td>722</td>      <td>recently</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>138</th>      <td>10</td>      <td>724</td>      <td>731</td>      <td>drinking</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>    </tr>    <tr>      <th>139</th>      <td>10</td>      <td>733</td>      <td>736</td>      <td>beer</td>      <td>B-behavior_alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>      <td>O</td>      <td>B-Alcohol</td>      <td>O</td>    </tr>    <tr>      <th>140</th>      <td>10</td>      <td>738</td>      <td>742</td>      <td>daily</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>    </tr>    <tr>      <th>141</th>      <td>10</td>      <td>743</td>      <td>743</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>142</th>      <td>11</td>      <td>745</td>      <td>747</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>143</th>      <td>11</td>      <td>749</td>      <td>754</td>      <td>smokes</td>      <td>B-behavior_tobacco</td>      <td>O</td>      <td>O</td>      <td>B-Smoking</td>      <td>O</td>      <td>O</td>      <td>B-Smoking</td>      <td>O</td>    </tr>    <tr>      <th>144</th>      <td>11</td>      <td>756</td>      <td>756</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Quantity</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Quantity</td>      <td>O</td>    </tr>    <tr>      <th>145</th>      <td>11</td>      <td>758</td>      <td>761</td>      <td>pack</td>      <td>B-behavior_tobacco</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Quantity</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Quantity</td>      <td>O</td>    </tr>    <tr>      <th>146</th>      <td>11</td>      <td>763</td>      <td>764</td>      <td>of</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>147</th>      <td>11</td>      <td>766</td>      <td>775</td>      <td>cigarettes</td>      <td>B-behavior_tobacco</td>      <td>O</td>      <td>O</td>      <td>B-Smoking</td>      <td>O</td>      <td>O</td>      <td>B-Smoking</td>      <td>O</td>    </tr>    <tr>      <th>148</th>      <td>11</td>      <td>777</td>      <td>777</td>      <td>a</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>      <td>O</td>      <td>B-Substance_Frequency</td>      <td>O</td>    </tr>    <tr>      <th>149</th>      <td>11</td>      <td>779</td>      <td>781</td>      <td>day</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Frequency</td>      <td>O</td>      <td>O</td>      <td>I-Substance_Frequency</td>      <td>O</td>    </tr>    <tr>      <th>150</th>      <td>11</td>      <td>782</td>      <td>782</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>151</th>      <td>12</td>      <td>784</td>      <td>786</td>      <td>She</td>      <td>O</td>      <td>O</td>      <td>B-Gender</td>      <td>B-Gender</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>152</th>      <td>12</td>      <td>788</td>      <td>790</td>      <td>had</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>153</th>      <td>12</td>      <td>792</td>      <td>794</td>      <td>DUI</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>B-Legal_Issues</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>154</th>      <td>12</td>      <td>796</td>      <td>797</td>      <td>in</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>155</th>      <td>12</td>      <td>799</td>      <td>803</td>      <td>April</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>156</th>      <td>12</td>      <td>805</td>      <td>807</td>      <td>and</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>157</th>      <td>12</td>      <td>809</td>      <td>811</td>      <td>was</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>158</th>      <td>12</td>      <td>813</td>      <td>815</td>      <td>due</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>159</th>      <td>12</td>      <td>817</td>      <td>818</td>      <td>to</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>160</th>      <td>12</td>      <td>820</td>      <td>824</td>      <td>court</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>161</th>      <td>12</td>      <td>826</td>      <td>829</td>      <td>this</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>162</th>      <td>12</td>      <td>831</td>      <td>834</td>      <td>week</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>    <tr>      <th>163</th>      <td>12</td>      <td>835</td>      <td>835</td>      <td>.</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>      <td>O</td>    </tr>  </tbody></table>